In [1]:
import numpy as np
import pandas as pd
import ast
import os
from pathlib import Path
from sqlalchemy import create_engine, MetaData

In [2]:
# Load DBs
manufacturers_engine = create_engine('sqlite:///manufacturers.db')
wallapop_engine = create_engine('sqlite:///wallapop_merged.db')

# print(manufacturers_engine.table_names())
# print(wallapop_engine.table_names())

In [3]:
# Create dataframes
df_manufacturers = pd.read_sql('manufacturer', manufacturers_engine)
df_cars = pd.read_sql('cars', wallapop_engine)

len(df_cars)
# print(df_manufacturers.head())
# print('-'*100)
# print(df_cars.head())

912

In [4]:
df_cars = df_cars.drop(columns=['Unnamed: 0', 'level_0', 'index', 'Financed Price', 'Warranty'])
# print(df_cars.head())

In [5]:
# Delete specific row
df_cars = df_cars[df_cars['Url Id'] != 'www0a0a']

In [6]:
# Remove trainling .0 , trailing € and . in between numbers
df_cars['Price'] = df_cars['Price'].replace(' €$', '', regex=True)
df_cars = df_cars.replace('\.0$', '', regex=True)
df_cars['Price'] = df_cars['Price'].replace('\.', '', regex=True)

# df_cars

In [7]:
# Convert empty strings to NaN
print(len(df_cars))
df_cars = df_cars.replace('', np.nan)
df_cars = df_cars.dropna(subset=['Mileage', 'Price', 'Gear', 'Doors', 'Seats', 'Fuel', 'City'])
print(len(df_cars))

911
684


In [8]:
# Check out if is any cell with a NaN value
# df_cars[df_cars.isna().any(axis=1)]

In [9]:
# Convert type to int
df_cars['Price'] = df_cars['Price'].astype(int)
df_cars['Mileage'] = df_cars['Mileage'].astype(int)
df_cars['Year'] = df_cars['Year'].astype(int)
df_cars['Doors'] = df_cars['Doors'].astype(int)
df_cars['Seats'] = df_cars['Seats'].astype(int)

# mileage_col = df_cars['Mileage'].tolist()
# print(sorted(mileage_col))

print(df_cars.columns)
df_cars.iloc[0:10][['Manufacturer', 'Model', 'Posted Date', 'Scraping Date']]

Index(['Item Id', 'Url Id', 'Manufacturer', 'Model', 'Price', 'Mileage',
       'Year', 'Fuel', 'Gear', 'Doors', 'Seats', 'Province', 'City',
       'Zip Code', 'Posted Date', 'Scraping Date', 'Url'],
      dtype='object')


,Manufacturer,Model,Posted Date,Scraping Date
2,Renault,Captur,2019-11-02,2019-11-02
4,SEAT,Ibiza,2019-11-02,2019-11-02
5,Peugeot,206,2019-11-02,2019-11-02
6,SEAT,Leon,2019-11-02,2019-11-02
8,Citroen,minicamper.pro,2019-11-02,2019-11-02
11,BMW,Serie 3,2019-11-03,2019-11-03
12,Ford,Focus,2019-11-03,2019-11-03
14,Volkswagen,Golf,2019-11-03,2019-11-03
15,Peugeot,206,2019-11-03,2019-11-03
17,Nissan,Trade,2019-11-03,2019-11-03


In [10]:
df_cars['Posted Date'] = pd.to_datetime(df_cars['Posted Date'])
df_cars['Scraping Date'] = pd.to_datetime(df_cars['Scraping Date'])

In [11]:
df_cars.dtypes

Item Id                  object
Url Id                   object
Manufacturer             object
Model                    object
Price                     int64
Mileage                   int64
Year                      int64
Fuel                     object
Gear                     object
Doors                     int64
Seats                     int64
Province                 object
City                     object
Zip Code                 object
Posted Date      datetime64[ns]
Scraping Date    datetime64[ns]
Url                      object
dtype: object

In [12]:
clean_wallapop = create_engine('sqlite:///clean_wallapop_ads.db')
df_cars.to_sql('cars', clean_wallapop)